In [70]:
import pandas as pd
import numpy as np
import pickle
from ksenticnet_kaist import ksenticnet

# 1. ksenticnet에서 유의어만 추출

In [90]:
word_dic = dict()

for key, value in ksenticnet.items():
    temp_value = []
    for v in value:
        if v.isalpha() and not v.isupper() and not v.islower():
            temp_value.append(v)

    if temp_value:
        word_dic[key] = temp_value

# 2. kwn (Korean WordNet)과의 병합

In [91]:
with open(r"../data/wordnet.pickle", "rb") as f:
    kwn = pickle.load(f)

for key, value in kwn.items():

    if word_dic.get(key): # key가 있다면
        value_1 = set(word_dic[key])
        value_2 = set(kwn[key])
        word_dic[key] = list(value_1 | value_2)

    elif len(value) > 1:
        word_dic[key] = value    

# 3. EDA

In [124]:
import kss
import random

In [143]:
def synonym_replacement(text, num_replacement):

    morphemes = kss.split_morphemes(text, drop_space=False)
    morphemes_copy = list(enumerate(morphemes.copy()))

    cnt_replacement = 0

    # 순서 섞기
    random.shuffle(morphemes_copy)

    for idx, (morph, pos) in morphemes_copy:

        if pos.startswith("N") or pos.startswith("V"):
            dict_value = word_dic.get(morph)

            if dict_value:
                replace_word = set(dict_value).pop()
                morphemes[idx] = (replace_word, pos)
                cnt_replacement += 1
            
        if cnt_replacement > num_replacement:
            break

    # 문서 재배치
    text = ""

    for morph, _ in morphemes:
        text += morph

    return text

In [163]:
text = "회사 동료 분들과 다녀왔는데 분위기도 좋고 음식도 맛있었어요 다만, 강남 토끼정이 강남 쉑쉑버거 골목길로 쭉 올라가야 하는데 다들 쉑쉑버거의 유혹에 넘어갈 뻔 했답니다"
t = synonym_replacement(text, 5)
tt = random_insertion(text, 5)

print(t)
print(tt)

기업 동료 분들과 다녀왔는데 분위기도 좋고 음식도 맛있었어요 다만, 강남 토끼노정이 강남 쉑쉑버거 소로로 쭉 올라가야 하는데 다들 쉑쉑버거의 유혹에 넘어갈 뻔 했답니다
회사 동료소로 분들과 다녀왔는데 분위기도 좋고 음식노정도 맛있었어요 다만, 강남 토끼정이 강남 쉑쉑버거 골목길로기업 쭉 올라가야 하는데 다들 쉑쉑버거의 유혹에 넘어갈 뻔 했답니다


In [162]:
def random_insertion(text, num_replacement):
    
    morphemes = kss.split_morphemes(text, drop_space=False)
    morphemes_copy = list(enumerate(morphemes.copy()))

    cnt_replacement = 0
    inserted = []

    # 순서 섞기
    random.shuffle(morphemes_copy)

    for idx, (morph, pos) in morphemes_copy:

        if pos.startswith("N") or pos.startswith("V"):
            dict_value = word_dic.get(morph)

            if dict_value:
                replace_word = set(dict_value).pop()
                rand_int = random.randint(0, len(morphemes_copy)) + random.random()
                inserted.append((rand_int, (replace_word, pos)))
                cnt_replacement += 1
            
        if cnt_replacement > num_replacement:
            break

    # 문서 재배치
    text = ""
    morphemes_copy.extend(inserted)
    morphemes_copy.sort()

    for _, (morph, _) in morphemes_copy:
        text += morph

    return text

In [ ]:
def random_deletetion(text, num_del):
    pass